In [21]:
import numpy as np

In [22]:
from aiida import load_profile

load_profile()

Profile<uuid='d106cde233334800a702a20040a6b629' name='adis'>

In [23]:
from pyiron_base import Project, job

In [24]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [25]:
from python_workflow_definition.aiida import load_workflow_json

In [26]:
from quantum_espresso_workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [27]:
workflow_json_filename = "pyiron_base_to_aiida_qe.json"

In [28]:
calculate_qe = job(_calculate_qe, output_key_lst=["energy", "volume", "structure"])
generate_structures = job(_generate_structures)
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

In [29]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [30]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

0it [00:00, ?it/s]

In [31]:
structure = get_bulk_structure(
    element="Al",
    a=4.05,
    cubic=True,
    pyiron_project=pr,
)

In [32]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
    pyiron_project=pr,
)

In [33]:
number_of_strains = 5
structure_lst = generate_structures(  # the generate_structures() function is not available in the workflow graph
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
    pyiron_project=pr,
    list_length=number_of_strains,
)

In [34]:
job_strain_lst = []
for i, structure_strain in enumerate(structure_lst):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": structure_strain,
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
        pyiron_project=pr,
    )
    job_strain_lst.append(calc_strain)

In [35]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
    pyiron_project=pr,
)

In [36]:
write_workflow_json(delayed_object=plot, file_name=workflow_json_filename)

In [37]:
!cat {workflow_json_filename}

───────┬────────────────────────────────────────────────────────────────────────
       │ File: pyiron_base_to_aiida_qe.json
───────┼────────────────────────────────────────────────────────────────────────
   1   │ {"nodes": {"0": "quantum_espresso_workflow.plot_energy_volume_curve", "
       │ 1": "python_workflow_definition.shared.get_list", "2": "quantum_espress
       │ o_workflow.calculate_qe", "3": "python_workflow_definition.shared.get_d
       │ ict", "4": "quantum_espresso_workflow.generate_structures", "5": "quant
       │ um_espresso_workflow.calculate_qe", "6": "python_workflow_definition.sh
       │ ared.get_dict", "7": "quantum_espresso_workflow.get_bulk_structure", "8
       │ ": "quantum_espresso_workflow.calculate_qe", "9": "python_workflow_defi
       │ nition.shared.get_dict", "10": "quantum_espresso_workflow.calculate_qe"
       │ , "11": "python_workflow_definition.shared.get_dict", "12": "quantum_es
       │ presso_workflow.calculate_qe", "13": "python_workflow_de

In [38]:
wg = load_workflow_json(file_name=workflow_json_filename)

In [39]:
result = wg.run()

03/21/2025 02:07:58 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/21/2025 02:08:00 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: tasks ready to run: pickle_node18,pickle_node19,pickle_node20,pickle_node21,pickle_node22,pickle_node23,pickle_node24,pickle_node25,pickle_node26,pickle_node27,pickle_node28,pickle_node29,pickle_node30,pickle_node31,pickle_node32
03/21/2025 02:08:00 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|run_tasks]: Run task: pickle_node18, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'strain_0'}


03/21/2025 02:08:01 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|run_tasks]: Run task: pickle_node19, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'value': 'mini'}


03/21/2025 02:08:03 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|run_tasks]: Run task: pickle_node20, type: PYTHONJOB


...

03/21/2025 02:09:38 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 15466
03/21/2025 02:09:45 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|update_task_state]: Task: get_dict7 finished.
03/21/2025 02:09:50 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/21/2025 02:09:52 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: tasks ready to run: calculate_qe6
03/21/2025 02:09:52 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|run_tasks]: Run task: calculate_qe6, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'input_dict': <PickledData: uuid: 9a6d8290-aebd-46a9-ab14-9b1c393070c8 (pk: 15470) : {'calculation': 'vc-relax', 'kpts': [3, 3, 3], 'pseudopotentials': {'Al': 'Al.pbe-n-kjpaw_psl.1.0.0.UPF'}, 'smearing': 0.02, 'structure': '{"immutable_id": null, "last_modified": null, "elements": ["Al"], "nelements": 1, "elements_ratios": [1.0], "chemical_formula_descriptive": "Al4", "chemical_formula_reduced": "Al", "chemical_formula_hill": null, "chemical_formula_anonymous": "A", "dimension_types": [1, 1, 1], "nperiodic_dimensions": 3, "lattice_vectors": [[4.05, 0.0, 0.0], [0.0, 4.05, 0.0], [0.0, 0.0, 4.05]], "space_group_symmetry_operations_xyz": null, "space_group_symbol_hall": null, "space_group_symbol_hermann_mauguin": null, "space_group_symbol_hermann_mauguin_extended": null, "space_group_it_number": null, "cartesian_site_positions": [[0.0, 0.0, 0.0], [0.0, 2.025, 2.025], [2.025, 0.0, 2.025], [2.025, 2.025, 0.0]], "nsites": 

03/21/2025 02:09:57 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 15477
03/21/2025 02:10:32 PM <3985753> aiida.parser.PythonJobParser: [WARNING] Found extra results that are not included in the output: dict_keys(['energy', 'volume'])
03/21/2025 02:10:38 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/21/2025 02:10:40 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: tasks ready to run: generate_structures5
03/21/2025 02:10:40 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|run_tasks]: Run task: generate_structures5, type: PYTHONJOB


------------------------------------------------------------
kwargs:  {'register_pickle_by_value': True, 'strain_lst': <List: uuid: 7e545e32-edc9-4efa-84ec-7222572fcdb7 (pk: 15443) value: [0.9, 0.95, 1.0, 1.05, 1.1]>, 'structure': <Str: uuid: 3bfe17a6-b064-4069-a2c5-4446ba0b0e99 (pk: 15481) value: {"immutable_id": null, "last_modified": null, "elements": ["Al"], "nelements": 1, "elements_ratios": [1.0], "chemical_formula_descriptive": "Al4", "chemical_formula_reduced": "Al", "chemical_formula_hill": null, "chemical_formula_anonymous": "A", "dimension_types": [1, 1, 1], "nperiodic_dimensions": 3, "lattice_vectors": [[4.045637391294585, 0.0, 0.0], [0.0, 4.045637391294585, 0.0], [0.0, 0.0, 4.045637391294585]], "space_group_symmetry_operations_xyz": null, "space_group_symbol_hall": null, "space_group_symbol_hermann_mauguin": null, "space_group_symbol_hermann_mauguin_extended": null, "space_group_it_number": null, "cartesian_site_positions": [[0.0, 0.0, 0.0], [0.0, 2.022818695647293, 2.0228

03/21/2025 02:10:45 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 15488
03/21/2025 02:10:52 PM <3985753> aiida.orm.nodes.process.calculation.calcjob.CalcJobNode: [WARNING] output parser returned exit code<11>: The process did not register a required output.
03/21/2025 02:11:01 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: Continue workgraph.
03/21/2025 02:11:03 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|continue_workgraph]: tasks ready to run: 
03/21/2025 02:11:06 PM <3985753> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [15268|WorkGraphEngine|is_workgraph_finished]: WorkGraph finished, but tasks: ['generate_structures5'] failed. Thus all their child tasks are skipped.


In [20]:
from IPython.display import Image, display

plot_task = [t for t in wg.tasks if t.name.startswith('plot_energy_volume_curve')][0]
plot_file = f"{plot_task.node.get_remote_workdir()}/evcurve.png"

display(Image(filename=str(plot_file)))


AttributeError: 'DynamicNode' object has no attribute 'node'